In [1]:
import pandas as pd
import os
import warnings

pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

shop = 'Bayer'

# Order Data cannot be extracted from Lazada Seller Center in one go, so we had to break it down into multiple excel files
# All excel files from the same shop are stored in one folder
folder = f'Lazada Seller Center/{shop}'
excel_files = os.listdir(folder)

# Loop over all the excel files in the folder and import only the "orderNumber" and "status" columns
dfs = {}
for file in excel_files:
    file_path = os.path.join(folder, file)
    df = pd.read_excel(file_path, usecols=["orderNumber", "status", "createTime"])
    dfs[file] = df

# The result will be one DataFrame with "orderNumber" and "status" columns for each excel file
for file, df in dfs.items():
    print(file)
    print("Number of rows:", len(df))

Lazada Bayer (March 16 - April 6).xlsx
Number of rows: 13511
          createTime      orderNumber     status
0  06 Apr 2024 23:58  815626858923070  confirmed
1  06 Apr 2024 23:36  821582920216964  confirmed
2  06 Apr 2024 23:36  821582920216964  confirmed
3  06 Apr 2024 23:01  815594271466239  confirmed
4  06 Apr 2024 22:06  815569891264333  confirmed 



In [2]:
# Merge into one DataFrame
sc_raw = pd.concat(dfs.values(), keys=dfs.keys(), ignore_index=True)
count_sc_raw = len(sc_raw)

# Remove duplicate entries
sc_unique = sc_raw.drop_duplicates(subset=['orderNumber'])
count_sc_unique = len(sc_unique)

print(f"SC Lazada {shop}")
print("Number of rows:", count_sc_raw)
print("Number of unique rows:", count_sc_unique)

SC Lazada Bayer
Number of rows: 13511
Number of unique rows: 4890


In [3]:
# Import Snowflake data into a DataFrame
snowflake_raw = pd.read_csv(f'Snowflake/Snowflake Lazada {shop}.csv', usecols=['PLFM_ORD_ID','ORD_STAT_END', 'ORD_STAT_PLFM', 'ORDER_DATE'])
count_snowflake_raw = len(snowflake_raw)

# Determine date range of the dataset
date_range = pd.date_range(start=snowflake_raw.loc[:, 'ORDER_DATE'].min(), end=snowflake_raw.loc[:, 'ORDER_DATE'].max())
sf_start_date = date_range[0].date()
sf_end_date = date_range[-1].date()

# Remove duplicates
snowflake_unique = snowflake_raw.drop_duplicates(subset=['PLFM_ORD_ID'], ignore_index=True)
count_snowflake_unique = len(snowflake_unique)

# Get non-end status rows
end_status = ['delivered','cancelled','returned']

# The ~ negates the condition 'merged_unique['ORD_STAT_END'].isin(end_status)'
snowflake_non_end_status = snowflake_unique[~snowflake_unique['ORD_STAT_END'].isin(end_status)]
count_snowflake_non_end_status = len(snowflake_non_end_status)

print(f"Snowflake Lazada {shop} ({sf_start_date} - {sf_end_date})")
print("Number of rows:", count_snowflake_raw)
print("Number of unique rows:", count_snowflake_unique)
print("Number of non-end status rows:", count_snowflake_non_end_status)

Snowflake Lazada Bayer (2024-03-17 - 2024-04-06)
Number of rows: 13140
Number of unique rows: 4751
Number of non-end status rows: 1065


In [7]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
merged_non_end_status = pd.merge(sc_unique, snowflake_non_end_status, left_on="orderNumber", right_on="PLFM_ORD_ID", how="inner")
count_merged_non_end_status = len(merged_non_end_status)
total = count_snowflake_unique

# Count the number of mismatched status
mismatched_status = merged_non_end_status[merged_non_end_status["status"] != merged_non_end_status["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100

print(f"Lazada {shop} SC vs Snowflake ({sf_start_date} - {sf_end_date})")
print(mismatched_status.head(10), '\n')
print("Number of rows:", total)
print("Number of mismatched non-end status:", count_mismatched_status)
print("Percent mismatched non-end status:", round(percent_mismatched_status, 2), "%")


Lazada Bayer SC vs Snowflake (2024-03-17 - 2024-04-06)
           createTime      orderNumber     status      PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
1   05 Apr 2024 21:23  821038510604358   canceled  821038510604358      pending        packed  2024-04-05
2   04 Apr 2024 23:58  814592688032785  delivered  814592688032785      shipped       shipped  2024-04-04
3   04 Apr 2024 23:57  820557741732785  delivered  820557741732785      shipped       shipped  2024-04-04
4   04 Apr 2024 23:51  820521394377084  confirmed  820521394377084      shipped       shipped  2024-04-04
5   04 Apr 2024 23:36  820554310484554  confirmed  820554310484554      shipped       shipped  2024-04-04
6   04 Apr 2024 20:04  820412732554676  delivered  820412732554676      shipped       shipped  2024-04-04
7   04 Apr 2024 19:22  814463404131910  delivered  814463404131910      shipped       shipped  2024-04-04
8   04 Apr 2024 19:18  820382137979866  confirmed  820382137979866      shipped       shipped  20